In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import sys
import json

In [7]:
def make_soup(start_url):
    """Beautiful parser."""
    s=Service('enter-base-path/chrome-driver/chromedriver')
    browser = webdriver.Chrome(service=s)
    browser.get(start_url)
    browser.implicitly_wait(100)
    html = browser.page_source
    bs = BeautifulSoup(html, 'lxml')
    return bs

In [8]:
# From: https://gist.github.com/vladignatyev/06860ec2040cb497f0f3
def progress(count, total, status=''):
    """Displays an inline progress bar"""
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush()

In [9]:
def get_links(start_url):
    """Fetch all the links to the next page"""
    count = 0
    links =set()
    links.add(start_url)
    soup = make_soup(start_url)
    all_links = soup.find('div', class_="paginator").find_all('a')
    total = len(all_links)
    for link in all_links:
        if link:
            links.add('https://www.influenster.com/'+link['href'])
        progress(count, total - 1, "Fetching links")
        count += 1
    return links

In [16]:
get_links('https://www.influenster.com/reviews/citi-simplicity-credit-card')

{'https://www.influenster.com//reviews/citi-simplicity-credit-card?review_page=10',
 'https://www.influenster.com//reviews/citi-simplicity-credit-card?review_page=2',
 'https://www.influenster.com//reviews/citi-simplicity-credit-card?review_page=3',
 'https://www.influenster.com//reviews/citi-simplicity-credit-card?review_page=4',
 'https://www.influenster.com//reviews/citi-simplicity-credit-card?review_page=5',
 'https://www.influenster.com//reviews/citi-simplicity-credit-card?review_page=6',
 'https://www.influenster.com//reviews/citi-simplicity-credit-card?review_page=7',
 'https://www.influenster.com//reviews/citi-simplicity-credit-card?review_page=8',
 'https://www.influenster.com//reviews/citi-simplicity-credit-card?review_page=9',
 'https://www.influenster.com/reviews/citi-simplicity-credit-card'}

In [17]:
# links to next page
url_links = []
for i in range(1,76):
    urls = 'https://www.influenster.com/reviews/citi-simplicity-credit-card?review_page={}'.format(i)
    url_links.append(urls)

In [18]:
len(url_links)

75

In [19]:
def get_data(start_url):
    """Fetch data from each link"""
    links = url_links; #get_links(start_url)
    count = 0
    total = len(links)
    content = []
    ratings = []
    
    for link in links:
            soup = make_soup(link)
            progress(count, total - 1, "Fetching data")
            count += 1
            for review in soup.find_all('div', class_="content-item-text review-text",itemprop="reviewBody"):
                content.append(review.text.strip('\n'))
            for rating in soup.find_all('div', class_ = 'avg-stars'):
                star = float(rating['data-stars'])
                if star.is_integer() is True:
                    ratings.append(star)
                
    review_dict = {'reviews':content}
    rating_dict = {'rating':ratings}
    
    with open('../raw_data/review_citi_simplicity.json', 'a') as f:
        json.dump(review_dict,f)
    with open('../raw_data/rating_citi_simplicity.json', 'a') as f:
        json.dump(rating_dict,f)

In [20]:
get_data('https://www.influenster.com/reviews/citi-simplicity-credit-card')

In [21]:
# save data as csv dataframe
df_review = pd.read_json('../raw_data/review_citi_simplicity.json')
df_rating = pd.read_json('../raw_data/rating_citi_simplicity.json')
df_review.reviews = df_review.reviews.str.replace(r"\n", "")
df_review['rating'] = df_rating
df_review['company'] = 'Citi Simplicity Card'
df_review.to_csv('../raw_data/citi_simplicity.csv', index = False)
df_review.tail()

,reviews,rating,company
738,Great card...,4,Citi Simplicity Card
739,I have use...,5,Citi Simplicity Card
740,Es muy bue...,5,Citi Simplicity Card
741,I like usi...,5,Citi Simplicity Card
742,Great cust...,5,Citi Simplicity Card
